In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import keras
import sklearn

# from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))

/Users/Kyelee/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
songs = pd.read_csv('SongCSV.csv', ',')

In [3]:
songs.columns

Index(['SongNumber', 'SongID', 'AlbumID', 'AlbumName', 'ArtistFamiliarity',
       'ArtistHotttnesss', 'ArtistID', 'ArtistLatitude', 'ArtistLocation',
       'ArtistLongitude', 'ArtistName', 'BarsConfidence', 'BarsStart',
       'BeatsConfidence', 'BeatsStart', 'Danceability', 'Duration',
       'EndOfFadeIn', 'Energy', 'KeySignature', 'KeySignatureConfidence',
       'Loudness', 'Mode', 'ModeConfidence', 'SectionsConfidence',
       'SectionsStart', 'SegmentsConfidence', 'SegmentsLoudnessMax',
       'SegmentsLoudnessMaxTime', 'SegmentsLoudnessMaxStart',
       'SegmentsPitches', 'SegmentsStart', 'SegmentsTimbre', 'SongHotttnesss',
       'TatumsConfidence', 'TatumsStart', 'Tempo', 'TimeSignature',
       'TimeSignatureConfidence', 'Title', 'Year'],
      dtype='object')

In [4]:
songs.shape
songs = songs.drop(columns = ['SongNumber', 'SongID', 'AlbumID', 'ArtistID'])

In [5]:
songs.iloc[0:4, :]

,AlbumName,ArtistFamiliarity,ArtistHotttnesss,ArtistLatitude,ArtistLocation,ArtistLongitude,ArtistName,BarsConfidence,BarsStart,BeatsConfidence,...,SegmentsStart,SegmentsTimbre,SongHotttnesss,TatumsConfidence,TatumsStart,Tempo,TimeSignature,TimeSignatureConfidence,Title,Year
0,b'Je Sais Que La Terre Est Plate (Deluxe)',0.557460,0.386152,46.71067,b'France',1.71819,b'Rapha\xc3\xabl',[0.179 0.373 0.127 0.015 0.012 0.119 0.083 0.0...,[ 0.52458 2.4444 4.3658 6.30006 8.2...,[0.886 0.725 0.748 0.721 0.784 0.417 0.421 0.5...,...,[ 0. 0.19188 0.60499 0.84939 1.1...,[[ 0. 171.13 9.469 ... 0.973 -10...,0.547953,[0. 0. 0. 0. 0. 0. 0. 0. ...,[4.2570000e-02 2.8357000e-01 5.2458000e-01 7.6...,124.059,4,0.372,b'Je Sais Que La Terre Est Plate',2008
1,b'Comme Vous',0.626958,0.434860,NaN,b'',NaN,b'Julie Zenatti',[0.716 0.306 0.015 0.572 0.529 0.353 0.206 0.9...,[ 8.69527 12.01758 14.81363 20.09003 22.6...,[0.773 0.247 0.402 0.115 0.092 0.436 0.524 0.4...,...,[ 0. 0.51379 0.89138 1.30912 1.6...,[[ 8.00000e-03 1.70986e+02 9.12600e+00 ... ...,0.475638,[0.365 0.321 0.29 0.264 0.238 0.217 0.208 0.1...,[ 0.59161 0.84042 1.08799 1.33555 1.5...,80.084,4,0.533,b'On Efface',2004
2,b'Watkins Ale - Music of the English Renaissa...,0.425724,0.000000,NaN,b'',NaN,b'The Baltimore Consort',[0.014 0.025 0.049 0.029 0.11 0.484 0.357 0.0...,[ 1.95344 6.35701 10.70579 15.07913 19.48322 ...,[0.506 0.467 0.45 0.682 0.341 0.406 0.383 0.5...,...,[ 0. 0.97365 1.34431 1.81351 2.19764 ...,[[ 1.337 145.247 -43.274 ... 16.283 -13...,NaN,[0.307 0.305 0.291 0.284 0.281 0.28 0.276 0.2...,[ 1.11537 1.39701 1.67864 1.95344 2.23097 ...,54.874,4,0.000,b'Howells Delight',0
3,"bDon't Worry Lady""""",0.611495,0.334520,NaN,b'',NaN,b'I Hate Sally',[0.095 0.147 0.157 0.191 0.115 0.222 0.275 0.0...,[ 0.39309 2.75109 5.13649 7.52978 9.9...,[0.862 0.509 0.011 0.39 0.479 0.422 0.459 0.4...,...,[0.0000000e+00 7.3060000e-02 3.6862000e-01 6.4...,[[ 14.832 -56.33 69.981 ... 92.933 29...,NaN,[0.731 0.628 0.526 0.359 0.287 0.246 0.21 0.1...,[1.1929000e-01 3.9309000e-01 6.5603000e-01 9.1...,77.150,3,0.369,b'Martha Served',2007


In [57]:
df = songs.values
X = np.column_stack((df[:, 0:2], df[:, 3:]))
y = songs.ArtistHotttnesss

print(X.shape)

(10000, 36)


In [58]:
import re
def List(string):
    return [float(i) for i in re.findall("\d+\.\d+", string)]

In [59]:
# helper function to get indices for data types 

def get_indices(X, row, data_type):
    indices = [] 
    for i in range(0, X.shape[1]):
        if type(X[row, i]) is data_type:
            indices.append(i)
    return indices

# getting columns with string lists that must be parsed

indices = [] 
for i in range(1, X.shape[1]):
    if type(X[0, i]) is str and len(X[0, i]) >= 30: # some larger, hopefully larger than all the real strings
        indices.append(i)
    
for i in range(len(indices)):
    for j in range(len(X)):
        value = List(X[j, indices[i]])
        X[j, indices[i]] = value
      
    
# getting rid of string columns for now
        
indices = get_indices(X, 0, str)

print(indices)

X = np.delete(X, indices, axis = 1)

[0, 3, 5]


In [60]:
X[33, :]

# [33, 3]

array([0.200729608076517, nan, nan, list([]), list([]), list([]),
       list([]), 0.0, 401.24036, 0.322, 0.0, 10, 0.596,
       -7.327000000000001, 0, 0.604,
       list([0.752, 0.583, 0.73, 0.909, 0.742, 0.577]),
       list([0.0, 3.2218, 5.6594, 3.9919007, 3.9934367, 3.9944426]),
       list([0.468, 0.645, 0.187, 0.748]),
       list([16.379, 21.445, 8.973, 12.078, 6.759]),
       list([0.04194, 0.02506, 0.0429, 0.0351, 0.03338]),
       list([28.162, 17.766, 16.133, 17.852]),
       list([0.296, 0.205, 0.166, 0.124, 0.806, 0.431, 0.247, 0.501, 0.298, 0.424, 0.688, 0.268, 0.188, 0.509, 0.165, 0.703, 0.891, 0.429, 0.435, 0.213, 0.158, 0.259, 0.047, 0.064, 0.073, 0.282]),
       list([0.0, 3.2218, 5.6594, 3.9919007, 3.9934367, 3.9944426]),
       list([171.13, 9.469, 0.973, 10.64, 7.228, 38.535, 120.797, 50.988, 8.131, 79.781, 14.035, 34.911, 315.928, 13.889, 22.748, 11.586, 2.008, 47.205, 58.417, 15.972, 9.524, 3.726, 6.048, 45.404, 49.778, 30.825, 11.422, 9.061, 24.071, 36.813, 29.4

In [61]:
# I'm going to go with the (temporary) solution of just taking the average of the list values to get a single 
# value for the model
import statistics

for row in range(len(X)):
    indices = get_indices(X, row, list)
    for i in range(len(indices)):
        for j in range(len(X)):
            try:
                value = mean(X[j, indices[i]])
                # print(value)
                X[j, indices[i]] = value
            except statistics.StatisticsError:
                X[j, indices[i]] = 0
                print('Error at value [' + str(j) + ' , ' + str(indices[i]) + ']')

Error at value [33 , 3]
Error at value [73 , 3]
Error at value [115 , 3]
Error at value [195 , 3]
Error at value [385 , 3]
Error at value [954 , 3]
Error at value [1028 , 3]
Error at value [1180 , 3]
Error at value [1247 , 3]
Error at value [1456 , 3]
Error at value [1748 , 3]
Error at value [1810 , 3]
Error at value [1969 , 3]
Error at value [2595 , 3]
Error at value [2610 , 3]
Error at value [2619 , 3]
Error at value [2659 , 3]
Error at value [2681 , 3]
Error at value [2838 , 3]
Error at value [2904 , 3]
Error at value [3131 , 3]
Error at value [3155 , 3]
Error at value [3193 , 3]
Error at value [3221 , 3]
Error at value [3280 , 3]
Error at value [3285 , 3]
Error at value [3683 , 3]
Error at value [3752 , 3]
Error at value [4031 , 3]
Error at value [4295 , 3]
Error at value [4410 , 3]
Error at value [4421 , 3]
Error at value [4582 , 3]
Error at value [4757 , 3]
Error at value [4771 , 3]
Error at value [4925 , 3]
Error at value [5176 , 3]
Error at value [5272 , 3]
Error at value [5383

Error at value [707 , 18]
Error at value [844 , 18]
Error at value [1044 , 18]
Error at value [1634 , 18]
Error at value [1726 , 18]
Error at value [1942 , 18]
Error at value [2197 , 18]
Error at value [2273 , 18]
Error at value [2675 , 18]
Error at value [3037 , 18]
Error at value [3492 , 18]
Error at value [3866 , 18]
Error at value [4107 , 18]
Error at value [4267 , 18]
Error at value [4758 , 18]
Error at value [5000 , 18]
Error at value [5469 , 18]
Error at value [5719 , 18]
Error at value [5818 , 18]
Error at value [6218 , 18]
Error at value [6736 , 18]
Error at value [6826 , 18]
Error at value [6866 , 18]
Error at value [7060 , 18]
Error at value [7886 , 18]
Error at value [7956 , 18]
Error at value [9793 , 18]
Error at value [9894 , 18]
Error at value [844 , 21]
Error at value [33 , 26]
Error at value [54 , 26]
Error at value [75 , 26]
Error at value [115 , 26]
Error at value [165 , 26]
Error at value [185 , 26]
Error at value [246 , 26]
Error at value [251 , 26]
Error at value 

Error at value [567 , 31]
Error at value [568 , 31]
Error at value [569 , 31]
Error at value [570 , 31]
Error at value [571 , 31]
Error at value [572 , 31]
Error at value [573 , 31]
Error at value [574 , 31]
Error at value [575 , 31]
Error at value [576 , 31]
Error at value [577 , 31]
Error at value [578 , 31]
Error at value [579 , 31]
Error at value [580 , 31]
Error at value [581 , 31]
Error at value [582 , 31]
Error at value [583 , 31]
Error at value [584 , 31]
Error at value [585 , 31]
Error at value [586 , 31]
Error at value [587 , 31]
Error at value [588 , 31]
Error at value [589 , 31]
Error at value [590 , 31]
Error at value [591 , 31]
Error at value [592 , 31]
Error at value [593 , 31]
Error at value [594 , 31]
Error at value [595 , 31]
Error at value [596 , 31]
Error at value [597 , 31]
Error at value [598 , 31]
Error at value [599 , 31]
Error at value [600 , 31]
Error at value [601 , 31]
Error at value [602 , 31]
Error at value [603 , 31]
Error at value [604 , 31]
Error at val

Error at value [2025 , 31]
Error at value [2026 , 31]
Error at value [2027 , 31]
Error at value [2028 , 31]
Error at value [2029 , 31]
Error at value [2030 , 31]
Error at value [2031 , 31]
Error at value [2032 , 31]
Error at value [2033 , 31]
Error at value [2034 , 31]
Error at value [2035 , 31]
Error at value [2036 , 31]
Error at value [2037 , 31]
Error at value [2038 , 31]
Error at value [2039 , 31]
Error at value [2040 , 31]
Error at value [2041 , 31]
Error at value [2042 , 31]
Error at value [2043 , 31]
Error at value [2044 , 31]
Error at value [2045 , 31]
Error at value [2046 , 31]
Error at value [2047 , 31]
Error at value [2048 , 31]
Error at value [2049 , 31]
Error at value [2050 , 31]
Error at value [2051 , 31]
Error at value [2052 , 31]
Error at value [2053 , 31]
Error at value [2054 , 31]
Error at value [2055 , 31]
Error at value [2056 , 31]
Error at value [2057 , 31]
Error at value [2058 , 31]
Error at value [2059 , 31]
Error at value [2060 , 31]
Error at value [2061 , 31]
E

Error at value [3524 , 31]
Error at value [3525 , 31]
Error at value [3526 , 31]
Error at value [3527 , 31]
Error at value [3528 , 31]
Error at value [3529 , 31]
Error at value [3530 , 31]
Error at value [3531 , 31]
Error at value [3532 , 31]
Error at value [3533 , 31]
Error at value [3534 , 31]
Error at value [3535 , 31]
Error at value [3536 , 31]
Error at value [3537 , 31]
Error at value [3538 , 31]
Error at value [3539 , 31]
Error at value [3540 , 31]
Error at value [3541 , 31]
Error at value [3542 , 31]
Error at value [3543 , 31]
Error at value [3544 , 31]
Error at value [3545 , 31]
Error at value [3546 , 31]
Error at value [3547 , 31]
Error at value [3548 , 31]
Error at value [3549 , 31]
Error at value [3550 , 31]
Error at value [3551 , 31]
Error at value [3552 , 31]
Error at value [3553 , 31]
Error at value [3554 , 31]
Error at value [3555 , 31]
Error at value [3556 , 31]
Error at value [3557 , 31]
Error at value [3558 , 31]
Error at value [3559 , 31]
Error at value [3560 , 31]
E

Error at value [5011 , 31]
Error at value [5012 , 31]
Error at value [5013 , 31]
Error at value [5014 , 31]
Error at value [5015 , 31]
Error at value [5016 , 31]
Error at value [5017 , 31]
Error at value [5018 , 31]
Error at value [5019 , 31]
Error at value [5020 , 31]
Error at value [5021 , 31]
Error at value [5022 , 31]
Error at value [5023 , 31]
Error at value [5024 , 31]
Error at value [5025 , 31]
Error at value [5026 , 31]
Error at value [5027 , 31]
Error at value [5028 , 31]
Error at value [5029 , 31]
Error at value [5030 , 31]
Error at value [5031 , 31]
Error at value [5032 , 31]
Error at value [5033 , 31]
Error at value [5034 , 31]
Error at value [5035 , 31]
Error at value [5036 , 31]
Error at value [5037 , 31]
Error at value [5038 , 31]
Error at value [5039 , 31]
Error at value [5040 , 31]
Error at value [5041 , 31]
Error at value [5042 , 31]
Error at value [5043 , 31]
Error at value [5044 , 31]
Error at value [5045 , 31]
Error at value [5046 , 31]
Error at value [5047 , 31]
E

Error at value [6554 , 31]
Error at value [6555 , 31]
Error at value [6556 , 31]
Error at value [6557 , 31]
Error at value [6558 , 31]
Error at value [6559 , 31]
Error at value [6560 , 31]
Error at value [6561 , 31]
Error at value [6562 , 31]
Error at value [6563 , 31]
Error at value [6564 , 31]
Error at value [6565 , 31]
Error at value [6566 , 31]
Error at value [6567 , 31]
Error at value [6568 , 31]
Error at value [6569 , 31]
Error at value [6570 , 31]
Error at value [6571 , 31]
Error at value [6572 , 31]
Error at value [6573 , 31]
Error at value [6574 , 31]
Error at value [6575 , 31]
Error at value [6576 , 31]
Error at value [6577 , 31]
Error at value [6578 , 31]
Error at value [6579 , 31]
Error at value [6580 , 31]
Error at value [6581 , 31]
Error at value [6582 , 31]
Error at value [6583 , 31]
Error at value [6584 , 31]
Error at value [6585 , 31]
Error at value [6586 , 31]
Error at value [6587 , 31]
Error at value [6588 , 31]
Error at value [6589 , 31]
Error at value [6590 , 31]
E

Error at value [8063 , 31]
Error at value [8064 , 31]
Error at value [8065 , 31]
Error at value [8066 , 31]
Error at value [8067 , 31]
Error at value [8068 , 31]
Error at value [8069 , 31]
Error at value [8070 , 31]
Error at value [8071 , 31]
Error at value [8072 , 31]
Error at value [8073 , 31]
Error at value [8074 , 31]
Error at value [8075 , 31]
Error at value [8076 , 31]
Error at value [8077 , 31]
Error at value [8078 , 31]
Error at value [8079 , 31]
Error at value [8080 , 31]
Error at value [8081 , 31]
Error at value [8082 , 31]
Error at value [8083 , 31]
Error at value [8084 , 31]
Error at value [8085 , 31]
Error at value [8086 , 31]
Error at value [8087 , 31]
Error at value [8088 , 31]
Error at value [8089 , 31]
Error at value [8090 , 31]
Error at value [8091 , 31]
Error at value [8092 , 31]
Error at value [8093 , 31]
Error at value [8094 , 31]
Error at value [8095 , 31]
Error at value [8096 , 31]
Error at value [8097 , 31]
Error at value [8098 , 31]
Error at value [8099 , 31]
E

Error at value [9456 , 31]
Error at value [9457 , 31]
Error at value [9458 , 31]
Error at value [9459 , 31]
Error at value [9460 , 31]
Error at value [9461 , 31]
Error at value [9462 , 31]
Error at value [9463 , 31]
Error at value [9464 , 31]
Error at value [9465 , 31]
Error at value [9466 , 31]
Error at value [9467 , 31]
Error at value [9468 , 31]
Error at value [9469 , 31]
Error at value [9470 , 31]
Error at value [9471 , 31]
Error at value [9472 , 31]
Error at value [9473 , 31]
Error at value [9474 , 31]
Error at value [9475 , 31]
Error at value [9476 , 31]
Error at value [9477 , 31]
Error at value [9478 , 31]
Error at value [9479 , 31]
Error at value [9480 , 31]
Error at value [9481 , 31]
Error at value [9482 , 31]
Error at value [9483 , 31]
Error at value [9484 , 31]
Error at value [9485 , 31]
Error at value [9486 , 31]
Error at value [9487 , 31]
Error at value [9488 , 31]
Error at value [9489 , 31]
Error at value [9490 , 31]
Error at value [9491 , 31]
Error at value [9492 , 31]
E

In [46]:
X[2651, :]

array([0.5451592956008632, nan, nan, 0.08713793103448275,
       119.48561068376068, 0.6844598698481562, 120.50262167019028, 0.0,
       245.21097999999998, 0.35700000000000004, 0.0, 7, 0.07, -7.545, 1,
       0.6859999999999999, 0.5846666666666667, 116.69412304292929,
       0.5263406113537118, 10.39137547408344, 0.04820546717171717,
       18.741725316455696, 0.2526, 116.69412304292929, 38.08808571428571,
       0.2658610492106501, 0.4789796195652174, 3.281365779704017, 117.975,
       4, 0.835, 0, 0], dtype=object)

In [62]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD, Adam, RMSprop, Adagrad
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from sklearn import model_selection
from sklearn import preprocessing

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.33, random_state=42)

model_in = X_train.shape[1] # number of input neurons

model = Sequential()
model.add(Dense(100, input_dim = model_in, activation = 'relu'))
model.add(Dense(20, activation = 'relu'))
model.add(Dense(20, activation = 'relu'))
model.add(Dense(1, activation = 'relu'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
6700/6700 [==============================] - 1s 127us/step - loss: nan - acc: 0.0000e+00


In [63]:
print(model.evaluate(X_test, y_test))

3300/3300 [==============================] - 0s 53us/step
[nan, 0.0]
